- 해당 프로그램 코딩은 임형욱이 개발 했으며 사용자 허락 없이 절대 상업용으로 사용해선 안됩니다.
- 또한 이 프로그램으로 특정 회사에 피해를 줄 경우
- 모든 책임은 프로그램 사용자에게 있음을 미리 공지합니다.
- 이 프로그램과 관련한 문의는 hwlim2010@naver.com으로 보내주세요^^

In [1]:
# 아시아경제 경상남도 뉴스 기사 크롤링하기
# https://www.asiae.co.kr/

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import numpy 
import pandas as pd  
import xlwt
import random
import os   # 폴더 생성하기 위해 필요함
import requests
import re

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print("아시아경제 경상남도 뉴스 기사 크롤링하기")
print("=" *100)
query = input('1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): ')

query_txt = '아시아경제' # 파일 저장 이름용 변수

cnt = int(input('2.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 15) # 한 페이지에 15건의 기사가 보여짐
print("크롤링 할 총 페이지 번호: ",page_cnt)


#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기 ★★★★★★
f_dir = input("3.파일을 저장할 폴더명만 쓰세요(기본값:c:\\Users\\):")
if f_dir == '' :
    f_dir="c:\\Users\\"

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'
    
#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
chrome_path = "C:/Users/pc/chromedriver/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

driver.get('https://www.asiae.co.kr/')
driver.maximize_window() 
time.sleep(2)

#Step 5. 자동으로 검색어 입력 후 조회하기
driver.find_element_by_link_text('검색하기').click()
time.sleep(1)
element = driver.find_element_by_xpath("/html/body/div[2]/div[1]/div[1]/div[1]/div/div/div[1]/form/fieldset/div/span/input")
driver.find_element_by_xpath("/html/body/div[2]/div[1]/div[1]/div[1]/div/div/div[1]/form/fieldset/div/span/input").click( )
element.send_keys(query)
element.send_keys("\n")
time.sleep(2)

#Step 6.뉴스 더보기 선택하기
driver.find_element_by_link_text('뉴스 더보기').click()
time.sleep(2)

#Step 7. 각 뉴스기사의 상세 정보를 수집합니다

click_count = 1

# 각 뉴스기사의 URL 주소를 추출하여 목록을 먼저 만든 후 상세 크롤링을 진행합니다.
# URL 수집하여 목록 만들기
url_list=[]
count = 0
#while ( click_count <= page_cnt + 1)  :
for a in range(1,page_cnt+1) :
    
    print("%s 번째 페이지에서 기사의 URL 정보를 추출합니다==============" %a)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    result_1 = soup.find('div','seo_news co_page')
    result_url_list=result_1.find('ul','news_lst').find_all('li')
        
    for i in result_url_list:
        url = i.find('a')['href']
        print(url)
        full_url = 'https:'+ url
        url_list.append(full_url)
            
        count += 1

        if count == cnt :
            break  

    print("%s 번째 페이지까지 %s건 기사 수집 완료 ====================" %(a,count))
    print("\n")
    
    a += 1
    try :
        driver.find_element_by_link_text("%s" %a).click()
    except :
        driver.find_element_by_link_text('다음 페이지로 이동').click()
        
    time.sleep(2)
    
# Step 8. 각 기사 별 상세 내역을 수집합니다
no = 1

cno2=[]
title2=[]
date2=[]
content2=[]

for i in range(0,len(url_list)):  
            
    print("\n")
    print("%s 번째 아시아경제 경상남도 기사 상세 정보입니다==========================" %no)

    driver.get(url_list[i])

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    content_all = soup.find('div','content')

    # 뉴스 등록 번호
    c_no = url_list[i].split('/')
    #cno_1 = c_no[4].split('?')
    cno_1 = c_no[4][8:]        
    print('1.뉴스등록번호: ',cno_1)
    cno2.append(cno_1)

    # 뉴스제목 추출
    title = content_all.find('div','area_title').find('h3').get_text().replace('\n','')
    print('2.뉴스제목:' , title.strip())
    title2.append(title.strip())

    # 뉴스작성일
    e_date = content_all.find('div','area_util').find('p','user_data').get_text()
    print('3.뉴스작성일: ',e_date.split(' ')[1])
    date2.append(e_date.split(' ')[1])
    
    # 뉴스내용
    content = str(content_all.find('div','article fb-quotable').find_all('p'))
    content1 = re.sub('<.+?>', '', content, 0).strip()
    print('4.뉴스내용: ',content1)
    content2.append(content1)
    
    driver.back()  # 이전 페이지로 돌아가기 ★★★★★
    
    time.sleep(2)
    
    no += 1

    if no > cnt :
        break

# Step 9. 출력 결과를 저장하기
# 출력 결과를 표(데이터 프레임) 형태로 만들기

asia = pd.DataFrame()

asia['뉴스등록번호'] = cno2
asia['뉴스제목'] = pd.Series(title2) 
asia['뉴스작성일'] = pd.Series(date2)
asia['뉴스내용'] = pd.Series(content2)
asia['뉴스URL'] = pd.Series(url_list)
       
# csv 형태로 저장하기
asia.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
asia.to_excel(fx_name , index=False)

driver.close( )

아시아경제 경상남도 뉴스 기사 크롤링하기
1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): 창원
2.크롤링 할 건수는 몇건입니까?: 200
크롤링 할 총 페이지 번호:  14
3.파일을 저장할 폴더명만 쓰세요(기본값:c:\Users\):
1 번째 페이지에서 기사의 URL 정보를 추출합니다==============
//www.asiae.co.kr/article/2021082517385106713
//www.asiae.co.kr/article/2021082516131733661
//www.asiae.co.kr/article/2021082516141836370
//www.asiae.co.kr/article/2021082510513353146
//www.asiae.co.kr/article/2021082513590190936
//www.asiae.co.kr/article/2021082510465919805
//www.asiae.co.kr/article/2021082514035327410
//www.asiae.co.kr/article/2021082511174437055
//www.asiae.co.kr/article/2021082509590761451
//www.asiae.co.kr/article/2021082507495784827
//www.asiae.co.kr/article/2021082419111756974
//www.asiae.co.kr/article/2021082414283156624
//www.asiae.co.kr/article/2021082413470270177
//www.asiae.co.kr/article/2021082416421193786
//www.asiae.co.kr/article/2021082414343171415
1 번째 페이지까지 15건 기사 수집 완료 ====================


2 번째 페이지에서 기사의 URL 정보를 추출합니다==============
//www.asiae.co.kr/article/20

//www.asiae.co.kr/article/2021080517434678567
//www.asiae.co.kr/article/2021080514132345239
//www.asiae.co.kr/article/2021080513500492462
//www.asiae.co.kr/article/2021080510375419802
//www.asiae.co.kr/article/2021080510594645809
//www.asiae.co.kr/article/2021080510434601239
//www.asiae.co.kr/article/2021080509554968261
//www.asiae.co.kr/article/2021080509264481136
//www.asiae.co.kr/article/2021080413542639782
//www.asiae.co.kr/article/2021080408181869321
//www.asiae.co.kr/article/2021080409525169134
//www.asiae.co.kr/article/2021080317292095182
//www.asiae.co.kr/article/2021080314532937639
//www.asiae.co.kr/article/2021080314355380126
//www.asiae.co.kr/article/2021080312421194126
11 번째 페이지까지 165건 기사 수집 완료 ====================


12 번째 페이지에서 기사의 URL 정보를 추출합니다==============
//www.asiae.co.kr/article/2021080311560815810
//www.asiae.co.kr/article/2021080310194929445
//www.asiae.co.kr/article/2021080221211633107
//www.asiae.co.kr/article/2021080219170549021
//www.asiae.co.kr/article/2021080



7 번째 아시아경제 경상남도 기사 상세 정보입니다==========================
1.뉴스등록번호:  14035327410
2.뉴스제목: 한국예술종합학교 총장에 김대진 음악원 교수
3.뉴스작성일:  2021.08.25
4.뉴스내용:  [문화체육관광부는 한국예술종합학교 제9대 총장에 김대진 음악원 교수를 임명했다고 25일 전했다. 임기는 4년이다. 임명에는 직선제 선거가 적용됐다. 학생 등 학내 구성원들의 요구에 따라 지난 6월 투표를 진행했다., 김 총장은 줄리어드 음대 출신의 피아노 연주자 겸 지휘자다. 수원시립교향악단과 창원시립교향악단 예술감독을 비롯해 KBS교향악단 여수음악제 감독, 삼성문화재단 이사 등을 역임하며 대중과 소통해왔다. 한국예술종합학교와는 개교 직후인 1994년부터 인연을 맺었다. 음악원 교수로 활동하며 손열음, 김선욱, 이진상, 문지영 등 세계적인 피아니스트를 지도했다. 황희 문체부 장관은 "내년 개교 30주년을 맞는 한국예술종합학교가 김 총장의 전문성과 함께 한 단계 더 비상할 것으로 기대한다"라고 말했다., &lt;ⓒ경제를 보는 눈, 세계를 보는 창 아시아경제(www.asiae.co.kr) 무단전재 배포금지&gt;]


8 번째 아시아경제 경상남도 기사 상세 정보입니다==========================
1.뉴스등록번호:  11174437055
2.뉴스제목: [부고]박관천(전 청와대 공직기강비서관실 행정관)씨 모친상
3.뉴스작성일:  2021.08.25
4.뉴스내용:  [▶김옥순씨 별세, 박관천 전 경정(전 청와대 공직기강비서관실 행정관) 모친상 =25일 오전 10시, 창원한마음병원 장례식장 특실, 발인 27일 오전 10시, 장지 경남 진주시 내동 소재 선영. ☎055-225-1200, &lt;ⓒ경제를 보는 눈, 세계를 보는 창 아시아경제(www.asiae.co.kr) 무단전재 배포금지&gt;]


9 번째 아시아경제 경상남도 기사 상세 정보입니다==========================
1.뉴스등록번



18 번째 아시아경제 경상남도 기사 상세 정보입니다==========================
1.뉴스등록번호:  09525324066
2.뉴스제목: 경남도, 확진자 12명 발생 … 기존 확진자와 접촉
3.뉴스작성일:  2021.08.24
4.뉴스내용:  [경남도는 24일 오전 10시 기준으로 확진자가 12명 발생했다고 밝혔다., [아시아경제 영남취재본부 박새얀 기자] 경상남도는 24일 오전 10시 기준으로 전날 오후 5시 이후 확진자가 12명 발생했다고 밝혔다., 지역별로는 창원 7명, 함안 2명, 통영·함양·산청 각 1명이다.	, 감염 경로별로는 도내 확진자 접촉 8명, 다른 지역 확진자 접촉 1명, 조사 중 3명이다., 이에 따라 도내 누적 확진자는 총 9466명(입원 1026명, 퇴원 8413명, 사망 27명)으로 늘었다., &lt;ⓒ경제를 보는 눈, 세계를 보는 창 아시아경제(www.asiae.co.kr) 무단전재 배포금지&gt;]


19 번째 아시아경제 경상남도 기사 상세 정보입니다==========================
1.뉴스등록번호:  09065310208
2.뉴스제목: 오늘도 폭우…남부·충청권·제주에 150mm 물폭탄
3.뉴스작성일:  2021.08.24
4.뉴스내용:  [절기상 '처서'이자 제12호 태풍 '오마이스'의 영향으로 비가 내린 23일 서울 광화문 거리에서 시민들이 우산을 쓴 채 발걸음을 재촉하고 있다./김현민 기자 kimhyun81@, [아시아경제 한진주 기자] 태풍 오마이스가 지나간 사이에 서해상에서 다가오는 저기압으로 화요일인 24일도 전국에 많은 비가 내리겠다. 중부지방과 남부지방에 시간당 최대 70mm 이상 쏟아진다., 기상청은 이날 아침부터 낮 사이에는 남부지방과 충청권, 제주도를 중심으로 돌풍과 천둥·번개를 동반한 시간당 50~70mm의 매우 강한 비가 내릴 것이라고 예보했다. 전라권과 경남권은 200mm 이상, 중부지방은 최대 150mm 가량 많은 비가 오겠다.	, 23일부터 이날 새벽 4시까지 내린 



27 번째 아시아경제 경상남도 기사 상세 정보입니다==========================
1.뉴스등록번호:  18592605750
2.뉴스제목: 경남 12개 시·군 61명 확진 … 5일째 두자리수
3.뉴스작성일:  2021.08.23
4.뉴스내용:  [[이미지출처=AP연합뉴스], [아시아경제 영남취재본부 강샤론 기자] 경남도는 22일 오후 5시 이후 도내에서 신종 코로나바이러스감염증(코로나19) 확진자 61명이 발생했다고 23일 밝혔다., 지역별로 창원 34명, 김해 12명, 진주·양산·함안·밀양·창녕 각 2명, 사천·거제·함양·거창·합천 각 1명이다.	, 해외 입국한 외국인인 창녕 확진자 1명을 제외하면 모두 지역감염이다., 이 중 42명은 도내 확진자와 접촉했다., 대부분 가족이나 지인, 직장동료 등을 매개로 확진됐다.	, 나머지 9명은 다른 지역 확진자와 접촉했고, 9명은 감염경로를 조사 중이다., 이러한 확진자 수는 지난 18일 109명 확진자 발생 이후 19일 90명, 20일 79명, 21일 57명, 22일 54명 등 이날까지 5일째 두 자릿수 확진자 수를 기록했다., 도내 누적 확진자는 9454명(입원 1014명, 퇴원 8413명, 사망 27명)으로 늘었다., &lt;ⓒ경제를 보는 눈, 세계를 보는 창 아시아경제(www.asiae.co.kr) 무단전재 배포금지&gt;]


28 번째 아시아경제 경상남도 기사 상세 정보입니다==========================
1.뉴스등록번호:  16293098968
2.뉴스제목: 태풍 '오마이스' 북상 제주항공편  결항 속출
3.뉴스작성일:  2021.08.23
4.뉴스내용:  [제주 국제공항 활주로. (사진=황정필 기자), [제주=아시아경제 호남취재본부 박창원 기자] 제12호 태풍 ‘오마이스’가 제주로 북상하는 가운데 23일 오후 제주를 연결하는 항공편 28편이 결항했다., 이날 한국공항공사 제주지역본부에 따르면, 이날 오전 11시 기준 제주국제공항을 기점으로 운항 계획됐던 국내선 440편(

...



139 번째 아시아경제 경상남도 기사 상세 정보입니다==========================
1.뉴스등록번호:  13375179114
2.뉴스제목: 공수처, '직권남용' 최재형 고발 사건 대검 이첩
3.뉴스작성일:  2021.08.06
4.뉴스내용:  [지난 5일 오후  국민의힘 대권주자인 최재형 전 감사원장이 경남 창원시 마산회원구 국립 3·15 민주 묘지를 참배하고 취재진 질문에 답하고 있다. [이미지출처=연합뉴스], [아시아경제 김대현 기자] 고위공직자범죄수사처(공수처)가 사법정의바로세우기시민행동(사세행)이 고발한 최재형 전 감사원장 사건을 대검찰청으로 넘겼다., 6일 사세행에 따르면 공수처는 지난달 28일 조희연 서울시교육감의 특별채용 의혹에 표적감사를 한 혐의(직권남용권리행사방해) 등으로 고발된 최 전 원장 사건을 대검에 이첩했다.	, 앞서 사세행은 조 교육감의 전교조 출신 해직교사들 특별채용 의혹과 관련해 최 전 원장이 지나치게 엄격한 잣대로 감사했다며 지난 6월28일 공수처에 고발장을 냈다. 감사원도 과거 퇴직자 10명에 대한 필기시험 면제 의혹이 있었지만, 조 교육감 사건만 과잉 감사를 진행했다는 취지다., 이날 사세행은 "야권 유력 대선 예비후보인 최 전 원장을 수사하는 것이 정치적으로 부담스러워 회피한다면 고위공직자범죄를 성역없이 엄정하게 수사하라는 설립 취지에 정면으로 반하는 일"이라고 비판했다., &lt;ⓒ경제를 보는 눈, 세계를 보는 창 아시아경제(www.asiae.co.kr) 무단전재 배포금지&gt;]


140 번째 아시아경제 경상남도 기사 상세 정보입니다==========================
1.뉴스등록번호:  11305291836
2.뉴스제목: 윤석열·최재형 견제 과열… 野도 '네거티브전'
3.뉴스작성일:  2021.08.06
4.뉴스내용:  [(왼쪽부터) 원희룡 전 제주도지사, 홍준표·하태경 국민의힘 의원 [사진= 윤동주 기자, 연합뉴스], [아시아경제 박준이 기자] 여당의 네거티브로 얼룩진 대선 경선을 치르



148 번째 아시아경제 경상남도 기사 상세 정보입니다==========================
1.뉴스등록번호:  17160204065
2.뉴스제목: 경남대, 최형두 국회의원과 지역대학 LINC+사업 간담회 개최
3.뉴스작성일:  2021.08.05
4.뉴스내용:  [경남대학교 산학협력관 5층 산학협력세미나실에서 열린 'LINC+사업단 간담회'에 최형두 국회의원(가운데)과 경남지역 4개 대학 LINC+사업단장들이 참석했다.(이미지출처=경남대), [아시아경제 영남취재본부 이상현 기자] 경남대학교 LINC+사업단은 최형두 국회의원(창원 마산합포구), 경남지역 4개 대학과 'LINC+사업단 간담회'를 개최했다고 5일 밝혔다., 이날 간담회에서는 지난 10년간 산학협력 생태계 조성의 성공적인 모델로 자리 잡은 LINC+사업의 지속가능성을 중점적으로, 대학별 산학협력 선도대학 육성사업의 성과와 현황, 사업 필요성 등이 논의됐다.	, 최 의원은 "지난 10년간 1단계, 2단계를 거친 LINC 사업의 긍정적인 성과를 거뒀다"며 "지역소멸 위기에도 지역대학이 이끌어온 지속 가능한 지역발전을 위해 3단계 사업을 신경 쓰겠다"고 말했다., 박은주 경남대 사업단장은 "대학별 지역 맞춤 특성화 모델 정착을 통한 선순환 구조가 구축됐다"며 "지역공동체가 지속 가능한 방향으로 나아가기 위한 전폭적인 지원이 더욱 필요한 시점"이라고 말했다., &lt;ⓒ경제를 보는 눈, 세계를 보는 창 아시아경제(www.asiae.co.kr) 무단전재 배포금지&gt;]


149 번째 아시아경제 경상남도 기사 상세 정보입니다==========================
1.뉴스등록번호:  17433775758
2.뉴스제목: 창원시, 도심하천·저수지 생태계교란 생물 퇴치 나서
3.뉴스작성일:  2021.08.05
4.뉴스내용:  [창원시가 지난 2일부터 생태계교란 생물 퇴치 사업을 추진하고 있다., [아시아경제 영남취재본부 박새얀 기자] 경남 창원시는 지난 2일부터 60일간 도심하천과 저수지 등



156 번째 아시아경제 경상남도 기사 상세 정보입니다==========================
1.뉴스등록번호:  10434601239
2.뉴스제목: 경북에도 대구 교회發 확산 불똥 60명 확진자 발생
3.뉴스작성일:  2021.08.05
4.뉴스내용:  [코로나19 확진자가 다수 발생한 대구의 한 교회 정문에 폐쇄명령서가 붙어 있다. [이미지출처=연합뉴스], [아시아경제 영남취재본부 김용우 기자] 대구 교회와 태권도장발(發) 집단감염 불똥이 옮겨붙는 등 경북도에 하루 60명의 신규 확진자가 나왔다., 지역별로는 경산 26명, 포항 24명, 경주 4명, 구미 2명, 칠곡 2명, 김천 1명, 안동 1명이다.	, 경산에서는 대구 교회 관련 접촉자 21명과 대구 태권도장 관련 접촉자 4명 지난 25일 의성 확진자의 접촉자 1명이 확진 판정을 받았다., 포항에서는 태국인 모임 관련 접촉자 14명, 대구 수성구 확진자의 접촉자 1명, 아시아에서 입국한 1명, 대구 교회 관련 접촉자 2명, 감염취약시설 선제검사자 1명 등이다., 경주에서는 확진자 접촉자 3명, 경남 창원 확진자의 접촉자 1명 등이 양성 판정을 받았다.	, 구미에서는 포항 태국 지인 모임 관련 접촉자 1명 등이다., 칠곡에서는 사업장 관련 1명과 대구 달서구 확진자의 접촉자 1명이 감염됐다., 김천에서는 3일 확진자의 접촉자 1명, 안동에서는 대구 교회 관련 접촉자 1명이 확진됐다., 경북도에서는 최근 1주일간 국내 236명, 주간 일일평균 33.7명의 신규 확진자가 발생했으며 3875명이 자가 격리 중이다., &lt;ⓒ경제를 보는 눈, 세계를 보는 창 아시아경제(www.asiae.co.kr) 무단전재 배포금지&gt;]


157 번째 아시아경제 경상남도 기사 상세 정보입니다==========================
1.뉴스등록번호:  09554968261
2.뉴스제목: 경남도, 확진자 17명 발생 … 창원 마트·유흥주점 등
3.뉴스작성일:  2021.08.05
4.뉴스내용:  [[아시아경제



163 번째 아시아경제 경상남도 기사 상세 정보입니다==========================
1.뉴스등록번호:  14532937639
2.뉴스제목: 이준석 “내년 지방선거 경남지사 후보 현역의원 공천할 수도”
3.뉴스작성일:  2021.08.03
4.뉴스내용:  [국민의힘 이준석 대표가 3일 경남도의회 대회의실에서 열린 '경남 언론인과의 간담회'에서 발언하고 있다. [이미지출처=연합뉴스], [아시아경제 영남취재본부 김용우 기자, 영남취재본부 이상현 기자] 공석인 경남도지사 후보와 관련 국민의힘 이준석 대표는 내년 지방선거에서 현역의원도 공천할 수 있다는 뜻을 밝혔다., 이 대표는 3일 경남도의회에서 가진 경남 언론인과의 간담회에서 현역의원 공천 가능성을 언급하며 “당헌·당규상 현역의원 공천 배제 규정은 존재하지 않는다”고 말했다.	, 이 대표는 “내년 대통령선거 이후 지방선거 환경이 크게 변할 것으로 예상된다”며 “현역의원이라도 공정한 경쟁 과정에서 최적 후보로 선택받는다면 큰 페널티를 둘 계획은 현재로서는 없다”고 강조했다., 이 대표는 공공기관의 혁신도시 이전 등 지방분권 정책에 대해서는 “절반의 성공이라고 생각한다”고 말했다., “공공기관이나 산업의 이전 이후에 추가로 지역에 시너지 효과가 나지 않았다고 개인적으로 생각한다”며 이같이 지적했다.	, 또 “비교우위 측면에서 경남 같은 경우는 아직 동서축도 그렇고, 남북축도 그렇고, 광역교통 인프라가 확보되지 않아 지역별로 어려움을 겪는 경우가 있는 걸로 파악하고 있다”면서 “그런 것에 대해 우리 당에서 예산 확보와 더불어 여러 가지 아이디어를 대선 공약에 넣을 수 있도록 하겠다”고 밝혔다., 이 대표는 부·울·경(동남권) 메가시티와 가덕도 신공항과 관련해서는 “메가시티 경우, 부·울·경 지역에 있어 수도권과 같은 큰 권역을 형성하는 것이 시너지 효과를 내는 것에는 아주 중요한 정책이 될 수 있으나, 반대로 수도권에서 그런 베드타운 성격으로 외곽지역이 겪고 있는 공동화 현상도 우려된다”고 했다., 

TimeoutException: Message: timeout: Timed out receiving message from renderer: 299.958
  (Session info: chrome=92.0.4515.159)


- **HTML 태그 제거 참고 : https://tncnsfs.tistory.com/entry/python-%ED%8C%8C%EC%9D%B4%EC%8D%AC-html-%ED%83%9C%EA%B7%B8-%EC%A0%9C%EA%B1%B0**